## Connect to the NHANES data resource using the HPDS Adapter

In [18]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from utils import get_multiIndex_variablesDict, joining_variablesDict_onCol

In [24]:
PICSURE_network_URL = "https://3.236.133.248/picsure"
resource_id = "d39ae852-e22e-45d5-a7b2-6056ac54580f"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()

client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)
resource.help()



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------
|  Resource UUID                       |  Resource Name                                  
+--------------------------------------+------------------------------------------------------
| d39ae852-e22e-45d5-a7b2-6056ac54580f
+---------------------------------

### A "resource" allows access to the Data Dictionary and Query Engine of the underlying  data source

In [25]:
resource.dictionary()

## Create a new query instance and use .help() to see what can be done with it

In [26]:
query = resource.query()
query.help()


        .select()       list of data fields to return from resource for each record
        .crosscounts()  list of data fields that cross counts will be calculated for
        .require()      list of data fields that must be present in all returned records
        .anyof()        list of data fields that records must be a member of at least one entry
        .studies()      list of studies that are selected that the query will run against
        .filter()       list of data fields and conditions that returned records satisfy
                  [ Filter keys exert an AND relationship on returned records      ]
                  [ Categorical values have an OR relationship on their key        ]
                  [ Numerical Ranges are inclusive of their start and end points   ]

        .getCount()             single count indicating the number of matching records
        .getCrossCount()        array indicating number of matching records per cross-count keys
        .getResults()     

### Use .help() to see how to add entries to the selection criteria

In [27]:
query.select().help()


            select().
              add("key")            add a single column to be returned in results
              add(["key1", "key2"]) add several columns to be returned in results
              delete("key")         delete a single column from the list of columns to return
              show()                lists all current columns that will be returned in results
              clear()               clears all values from the select list
            


### Use a chained of commands on the dictionary object to include all labs in the query's selection list

In [28]:
query.select().add(resource.dictionary().find("disease").keys())

### Use the .show() command to see what is currently selected for the query

In [29]:
query.show()

.__________[ Query.select()  Settings ]_____________________________________________________________________________________________________________________
| _key__________________________________________________________________________________________________________________________
|  \\questionnaire\\disease\\any_cad\\                                                                                              |
|  \\questionnaire\\disease\\skin_cancer_self_report\\                                                                              |
|  \\questionnaire\\disease\\stomach_cancer_self_report\\                                                                           |
|  \\questionnaire\\disease\\brain_cancer_self_report\\                                                                             |
|  \\questionnaire\\disease\\other_cancer_self_report\\                                                                             |
|  \\questionnaire\\disease\\Any CAD (self r

### Clear the current selection list and add only "\\laboratory\\hormone\\" results

In [30]:
query.select().clear()
query.select().add(resource.dictionary().find("\\disease\\").keys())
query.select().show()

cleared list
| _key__________________________________________________________________________________________________________________________
|  \\questionnaire\\disease\\any_cad\\                                                                                              |
|  \\questionnaire\\disease\\skin_cancer_self_report\\                                                                              |
|  \\questionnaire\\disease\\stomach_cancer_self_report\\                                                                           |
|  \\questionnaire\\disease\\brain_cancer_self_report\\                                                                             |
|  \\questionnaire\\disease\\other_cancer_self_report\\                                                                             |
|  \\questionnaire\\disease\\Any CAD (self report)\\                                                                                |
|  \\questionnaire\\disease\\larynx_cancer_self_report

### Run some commands on the dictionary to find and then filter the query by gender 

In [34]:
disease = resource.dictionary().find("disease")
disease.keys()

['\\questionnaire\\disease\\any_cad\\',
 '\\questionnaire\\disease\\skin_cancer_self_report\\',
 '\\questionnaire\\disease\\stomach_cancer_self_report\\',
 '\\questionnaire\\disease\\brain_cancer_self_report\\',
 '\\questionnaire\\disease\\other_cancer_self_report\\',
 '\\questionnaire\\disease\\Any CAD (self report)\\',
 '\\questionnaire\\disease\\larynx_cancer_self_report\\',
 '\\questionnaire\\disease\\mouth_cancer_self_report\\',
 '\\questionnaire\\disease\\Any Diabetes (FBG >= 126 mg per dL or self-report)\\',
 '\\questionnaire\\disease\\esophagus_cancer_self_report\\',
 '\\questionnaire\\disease\\other_skin_cancer_self_report\\',
 '\\questionnaire\\disease\\Ever asthma?\\',
 '\\questionnaire\\disease\\testis_cancer_self_report\\',
 '\\questionnaire\\disease\\cervix_cacner_self_report\\',
 '\\questionnaire\\disease\\Ever rheumatoid arthritis?\\',
 '\\questionnaire\\disease\\kidney_cancer_self_report\\',
 '\\questionnaire\\disease\\lung_cancer_self_report\\',
 '\\questionnaire\\dis

In [35]:
disease.DataFrame()

,categorical,categoryValues,patientCount,observationCount,HpdsDataType,min,max
KEY,,,,,,,
\questionnaire\disease\any_cad\,True,"[No, Yes]",4891,4891,phenotypes,NaN,NaN
\questionnaire\disease\skin_cancer_self_report\,True,"[No, Yes]",4542,4542,phenotypes,NaN,NaN
\questionnaire\disease\stomach_cancer_self_report\,True,"[No, Yes]",4480,4480,phenotypes,NaN,NaN
\questionnaire\disease\brain_cancer_self_report\,True,"[No, Yes]",4477,4477,phenotypes,NaN,NaN
\questionnaire\disease\other_cancer_self_report\,True,"[No, Yes]",4494,4494,phenotypes,NaN,NaN
\questionnaire\disease\Any CAD (self report)\,True,"[No, Yes]",4859,4859,phenotypes,NaN,NaN
\questionnaire\disease\larynx_cancer_self_report\,True,"[No, Yes]",4481,4481,phenotypes,NaN,NaN
\questionnaire\disease\mouth_cancer_self_report\,True,"[No, Yes]",4477,4477,phenotypes,NaN,NaN
\questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\,True,"[No, Yes]",9451,9451,phenotypes,NaN,NaN


### Add a filter on our query to filter results to only female subjects

In [9]:
query.filter().help()


            filter().
              add("key", value)                  - or -
              add("key", "value")               filter to records with KEY column that equals VALUE
              add("key", ["value1", "value2"])  filter to records with KEY column equalling one value within the given list
              add("key", start, end)            filter to records with KEY column value between START and END (inclusive)
                                                    start -or- end may be set to None to filter by a max or min value
              delete("key")                     delete a filter from the list of filters
              show()                            lists all current filters that results records must satisfy
              clear()                           clears all values from the filters list
            


In [185]:
query.filter().add("\\disease\\")
query.show()
disdf = query.getResultsDataFrame()
disdf

ERROR: All Filter.add()'s must specify matching value, values, or range
.__________[ Query.select()  Settings ]_____________________________________________________________________________________________________________________
| _key__________________________________________________________________________________________________________________________
|  \\questionnaire\\disease\\any_cad\\                                                                                              |
|  \\questionnaire\\disease\\skin_cancer_self_report\\                                                                              |
|  \\questionnaire\\disease\\stomach_cancer_self_report\\                                                                           |
|  \\questionnaire\\disease\\brain_cancer_self_report\\                                                                             |
|  \\questionnaire\\disease\\other_cancer_self_report\\                                                   

,Patient ID,\questionnaire\disease\,\questionnaire\disease\Any CAD (self report)\,\questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\,\questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\,\questionnaire\disease\Any cancer or malignancy?\,\questionnaire\disease\Any family with heart attack or angina?\,\questionnaire\disease\Current asthma?\,\questionnaire\disease\Ever arthritis?\,\questionnaire\disease\Ever asthma?\,...,\questionnaire\disease\ovarian_cancer_self_report\,\questionnaire\disease\pancreatic_cancer_self_report\,\questionnaire\disease\prostate_cancer_self_report\,\questionnaire\disease\rectum_cancer_self_report\,\questionnaire\disease\skin_cancer_self_report\,\questionnaire\disease\soft_cancer_self_report\,\questionnaire\disease\stomach_cancer_self_report\,\questionnaire\disease\testis_cancer_self_report\,\questionnaire\disease\thyroid_cancer_self_report\,\questionnaire\disease\uterine_cancer_self_report\
0,1,NaN,No,No,No,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
1,2,NaN,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,112.85,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,5,NaN,NaN,No,No,NaN,NaN,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9995,9996,NaN,NaN,No,NaN,NaN,NaN,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,9997,NaN,No,No,Yes,No,Yes,No,Yes,No,...,No,No,No,No,No,No,No,No,No,No
9997,9998,NaN,NaN,No,NaN,NaN,NaN,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## View our query's criterion as it currently stands

In [282]:
d = resource.dictionary()
criteria = []
#criteria.extend(d.find("\\AGE\\").keys())
criteria.extend(d.find("\\disease\\").keys())

#criteria.extend(d.find("lung").keys())
#criteria.extend(d.find("blood pressure").keys())
#bp_dictionary_search = dictionary.find("blood pressure").DataFrame()
import pandas as pd
query_unified = resource.query()
query_unified.require().add(criteria)
df_mf = query_unified.getResultsDataFrame()
dfmf = df_mf[(df_mf.values == "Yes")] # & (df_mf.values == "Yes")]
dfmf

,Patient ID,\questionnaire\disease\,\questionnaire\disease\Any CAD (self report)\,\questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\,\questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\,\questionnaire\disease\Any cancer or malignancy?\,\questionnaire\disease\Any family with heart attack or angina?\,\questionnaire\disease\Current asthma?\,\questionnaire\disease\Ever arthritis?\,\questionnaire\disease\Ever asthma?\,...,\questionnaire\disease\ovarian_cancer_self_report\,\questionnaire\disease\pancreatic_cancer_self_report\,\questionnaire\disease\prostate_cancer_self_report\,\questionnaire\disease\rectum_cancer_self_report\,\questionnaire\disease\skin_cancer_self_report\,\questionnaire\disease\soft_cancer_self_report\,\questionnaire\disease\stomach_cancer_self_report\,\questionnaire\disease\testis_cancer_self_report\,\questionnaire\disease\thyroid_cancer_self_report\,\questionnaire\disease\uterine_cancer_self_report\
3,31,43.35,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
3,31,43.35,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,35,101.82,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,35,101.82,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
5,42,90.65,No,No,No,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,9911,83.72,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
563,9925,17.11,No,No,Yes,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
563,9925,17.11,No,No,Yes,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
563,9925,17.11,No,No,Yes,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No


In [283]:
duplicateRowsDF = dfmf[dfmf.duplicated()]
duplicateRowsDF

,Patient ID,\questionnaire\disease\,\questionnaire\disease\Any CAD (self report)\,\questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\,\questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\,\questionnaire\disease\Any cancer or malignancy?\,\questionnaire\disease\Any family with heart attack or angina?\,\questionnaire\disease\Current asthma?\,\questionnaire\disease\Ever arthritis?\,\questionnaire\disease\Ever asthma?\,...,\questionnaire\disease\ovarian_cancer_self_report\,\questionnaire\disease\pancreatic_cancer_self_report\,\questionnaire\disease\prostate_cancer_self_report\,\questionnaire\disease\rectum_cancer_self_report\,\questionnaire\disease\skin_cancer_self_report\,\questionnaire\disease\soft_cancer_self_report\,\questionnaire\disease\stomach_cancer_self_report\,\questionnaire\disease\testis_cancer_self_report\,\questionnaire\disease\thyroid_cancer_self_report\,\questionnaire\disease\uterine_cancer_self_report\
3,31,43.35,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,35,101.82,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
6,68,37.26,No,No,Yes,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
6,68,37.26,No,No,Yes,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
6,68,37.26,No,No,Yes,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,9903,61.45,No,No,Yes,No,Yes,Yes,No,Yes,...,No,No,No,No,No,No,No,No,No,No
559,9903,61.45,No,No,Yes,No,Yes,Yes,No,Yes,...,No,No,No,No,No,No,No,No,No,No
562,9911,83.72,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
563,9925,17.11,No,No,Yes,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No


In [284]:
dfmfs=dfmf.transpose()
dfmfs

,3,3,4,4,5,6,6,6,6,7,...,559,559,559,561,562,562,563,563,563,565
Patient ID,31,31,35,35,42,68,68,68,68,80,...,9903,9903,9903,9907,9911,9911,9925,9925,9925,9935
\questionnaire\disease\,43.35,43.35,101.82,101.82,90.65,37.26,37.26,37.26,37.26,113.21,...,61.45,61.45,61.45,56.99,83.72,83.72,17.11,17.11,17.11,82.1
\questionnaire\disease\Any CAD (self report)\,No,No,No,No,No,No,No,No,No,Yes,...,No,No,No,No,No,No,No,No,No,No
\questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
\questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,...,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,No
\questionnaire\disease\Any cancer or malignancy?\,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
\questionnaire\disease\Any family with heart attack or angina?\,No,No,No,No,Yes,Yes,Yes,Yes,Yes,No,...,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes
\questionnaire\disease\Current asthma?\,No,No,No,No,No,No,No,No,No,No,...,Yes,Yes,Yes,No,No,No,No,No,No,No
\questionnaire\disease\Ever arthritis?\,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
\questionnaire\disease\Ever asthma?\,No,No,No,No,No,No,No,No,No,No,...,Yes,Yes,Yes,No,No,No,No,No,No,No


In [287]:
dfmfs=dfmf.transpose()
dfmfs
# Iterate over the index range from o to max number of columns in dataframe
for index in range(dfmfs.shape[1]):
    print('Column Number : ', index)
    # Select column by index position using iloc[]
    columnSeriesObj = dfmfs.iloc[: , index]
    columnSeriesObj
    print('Column Yes Count : ', len(columnSeriesObj[(columnSeriesObj.values=="Yes")]))
    print('Column Yes Count : ', columnSeriesObj[(columnSeriesObj.values=="Yes")])

Column Number :  0
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 3, dtype: object
Column Number :  1
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 3, dtype: object
Column Number :  2
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 4, dtype: object
Column Number :  3
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 4, dtype: object


Column Yes Count :  1
Column Yes Count :  \questionnaire\disease\Any family with heart attack or angina?\    Yes
Name: 162, dtype: object
Column Number :  185
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Current asthma?\    Yes
\questionnaire\disease\Ever asthma?\       Yes
Name: 163, dtype: object
Column Number :  186
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Current asthma?\    Yes
\questionnaire\disease\Ever asthma?\       Yes
Name: 163, dtype: object
Column Number :  187
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 166, dtype: object
Column Number :  188
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 166, dtyp

Column Number :  351
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 298, dtype: object
Column Number :  352
Column Yes Count :  1
Column Yes Count :  \questionnaire\disease\Any family with heart attack or angina?\    Yes
Name: 299, dtype: object
Column Number :  353
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 300, dtype: object
Column Number :  354
Column Yes Count :  2
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 300, dtype: object
Column Number :  355
Column Yes Count :  1
Column Yes Count :  \questionnaire\d

Column Yes Count :  3
Column Yes Count :  \questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\     Yes
\questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 449, dtype: object
Column Number :  535
Column Yes Count :  3
Column Yes Count :  \questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\     Yes
\questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\any_chronic_disease\                                    Yes
Name: 449, dtype: object
Column Number :  536
Column Yes Count :  5
Column Yes Count :  \questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\    Yes
\questionnaire\disease\Any family with heart attack or angina?\                Yes
\questionnaire\disease\Current asthma?\                                        Yes
\questionnaire\disease\Ever asthma?

In [281]:
query.getCount()

9999

,Patient ID,\questionnaire\disease\,\questionnaire\disease\Any CAD (self report)\,\questionnaire\disease\Any Diabetes (FBG >= 126 mg per dL or self-report)\,\questionnaire\disease\Any Hypertension? (BP >= 139 per 90 or self report)\,\questionnaire\disease\Any cancer or malignancy?\,\questionnaire\disease\Any family with heart attack or angina?\,\questionnaire\disease\Current asthma?\,\questionnaire\disease\Ever arthritis?\,\questionnaire\disease\Ever asthma?\,...,\questionnaire\disease\ovarian_cancer_self_report\,\questionnaire\disease\pancreatic_cancer_self_report\,\questionnaire\disease\prostate_cancer_self_report\,\questionnaire\disease\rectum_cancer_self_report\,\questionnaire\disease\skin_cancer_self_report\,\questionnaire\disease\soft_cancer_self_report\,\questionnaire\disease\stomach_cancer_self_report\,\questionnaire\disease\testis_cancer_self_report\,\questionnaire\disease\thyroid_cancer_self_report\,\questionnaire\disease\uterine_cancer_self_report\
0,1,NaN,No,No,No,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
1,2,NaN,No,No,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,112.85,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,5,NaN,NaN,No,No,NaN,NaN,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9995,9996,NaN,NaN,No,NaN,NaN,NaN,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,9997,NaN,No,No,Yes,No,Yes,No,Yes,No,...,No,No,No,No,No,No,No,No,No,No
9997,9998,NaN,NaN,No,NaN,NaN,NaN,No,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# You can even work with multiple queries at the same time without having them accidently interact with each other!

In [15]:
gender = resource.dictionary().find("sex")
gender.help()
gender.DataFrame()
query.filter().add("\\demographics\\SEX\\", ["female"])
query.getCount()
query.getResultsDataFrame()

query2 = resource.query()
query3 = resource.query()


query2.select().add(resource.dictionary().find("\\demographics\\AGE").keys())
query3.select().add(resource.dictionary().find("\\demographics\\AGE").keys())


query2.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), min=60)
query3.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), max=20)
#query2.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), 60, None)
#query3.filter().add(resource.dictionary().find("\\demographics\\AGE").keys(), None, 20)

too_old = query2.getResultsDataFrame()
too_young = query3.getResultsDataFrame()

In [16]:
too_young.count()

Patient ID            21163
\demographics\AGE\    21163
dtype: int64

In [17]:
too_old.count()

Patient ID            7177
\demographics\AGE\    7177
dtype: int64